In [1]:
import pandas as pd
import numpy as np
import os
from ctgan import CTGANSynthesizer
from ctgan import load_demo
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import make_scorer
from sdv.evaluation import evaluate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from time import time
from sklearn.utils.fixes import loguniform
from sklearn.svm import SVC

# Cognitive Workload Dataset

In [2]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized')

In [3]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized')

li = []
for filename in filenames:
    dataframe=pd.read_csv(os.path.join(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized',filename))
    li.append(dataframe)

data = pd.concat(li, axis=0, ignore_index=True)

In [4]:
columns = [ 'HR_mean', 'HR_std', 'HR_min', 'HR_max', 'HRV_RMSSD',
       'HRV_MeanNN', 'HRV_SDNN', 'HRV_CVNN', 'HRV_CVSD', 'HRV_MedianNN',
       'HRV_MadNN', 'HRV_MCVNN', 'HRV_pNN50', 'HRV_pNN20', 'HRV_HF', 'HRV_VHF',
       'HRV_HFn', 'HRV_LFHF', 'HRV_SD1', 'HRV_SD2', 'HRV_SD1SD2', 'RSP_mean',
       'RSP_std', 'RSP_min', 'RSP_max', 'RRV_RMSSD', 'RRV_MeanBB', 'RRV_SDBB',
       'RRV_SD1', 'RRV_SD2', 'RRV_SD2SD1']

In [5]:
gp = data["ID"].values

In [6]:
data1= data[columns]
X=data1[columns]
y=data['label']

In [7]:
logo = LeaveOneGroupOut()
logo.get_n_splits(X, y, gp)

28

## Hyperparameter Tuning for Random Forest 

In [8]:
# GAN parameters
from sklearn.utils.fixes import loguniform
param_dist = {
    "max_depth": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    "max_features": ['auto', 'sqrt'],
    "bootstrap": [True, False],
    "min_samples_leaf": [1, 2, 4],
    "min_samples_split": [2, 5, 10],
    "n_estimators": [50,100, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

n_iter_search = 150
random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=n_iter_search,verbose=5, cv=logo
)

In [9]:
# code taken from https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py
start = time()
random_search.fit(X, y, gp)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")
            
            

report(random_search.cv_results_)

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=[ 4  4  4  4  4  4  5  5  5  5  5  5  7  7  7  7  7  7  6  6  6  6  6  6
 22 22 22 22 22 22 23 23 23 23 23 23 35 35 35 35 35 35 21 21 21 21 21 21
 20 20 20 20 20 20 34 34 34 34 34 30 30 30 30 30 24 24 24 24 24 31 31 31
 31 31 31 19 19 19 19 19 27 27 27 27 27 27 33 33 33 33 33 33 32 32 32 32
 32 32 26 26 26 26 26 17 17 17 17 17 17 16 16 16 16 16 16 28 28 28 28 28
 29 29 29 29 29 29 11 11 11 11 11 11 10 10 10 10 10 10 12 12 12 12 12 12
 13 13 13 13 13  8  8  8  8  8  8  9  9  9  9  9  9] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Fitting 28 folds for each of 150 candidates, totalling 4200 fits
[CV 1/28] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   0.9s
[CV 2/28] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   0.9s
[CV 3/28] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   0.9s
[CV 4/28] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   0.9s
[CV 5/28] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   0.9s
[CV 6/28] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   0.9s
[CV 7/28] END bootstrap=False, max_depth=60, max_features=sqrt, min_sam

[CV 2/28] END bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   0.8s
[CV 3/28] END bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   0.8s
[CV 4/28] END bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   0.8s
[CV 5/28] END bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   0.8s
[CV 6/28] END bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   0.8s
[CV 7/28] END bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   0.8s
[CV 8/28] END bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; tota

[CV 3/28] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   0.6s
[CV 4/28] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   0.6s
[CV 5/28] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   0.6s
[CV 6/28] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   0.6s
[CV 7/28] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   0.6s
[CV 8/28] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   0.6s
[CV 9/28] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=

[CV 5/28] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.6s
[CV 6/28] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.6s
[CV 7/28] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.6s
[CV 8/28] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.6s
[CV 9/28] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.6s
[CV 10/28] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.6s
[CV 11/28] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.

...

[CV 26/28] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1000; total time=   0.7s
[CV 27/28] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1000; total time=   0.7s
[CV 28/28] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1000; total time=   0.7s
[CV 1/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 2/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 3/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 4/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV

[CV 28/28] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   0.1s
[CV 1/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV 2/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV 3/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV 4/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV 5/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV 6/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV 7/28]

[CV 2/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 3/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 4/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 5/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 6/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 7/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 8/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 9/

[CV 4/28] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 5/28] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 6/28] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 7/28] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 8/28] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 9/28] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 10/28] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 1

[CV 6/28] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 7/28] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 8/28] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 9/28] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 10/28] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 11/28] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 12/28] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 13/28]

## Hyperparameter Tuning for Linear SVM 

In [10]:
param_dist = {
    "C": loguniform(1e0, 1e3),
    "gamma": loguniform(1e-4, 1e-3),
    "kernel": ["linear"],
    "class_weight": ["balanced", None],
}

n_iter_search = 150

random_search = RandomizedSearchCV(
    SVC(), param_distributions=param_dist, n_iter=n_iter_search, verbose=5, cv=logo
)

In [11]:
start = time()
random_search.fit(X, y, gp)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


report(random_search.cv_results_)

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=[ 4  4  4  4  4  4  5  5  5  5  5  5  7  7  7  7  7  7  6  6  6  6  6  6
 22 22 22 22 22 22 23 23 23 23 23 23 35 35 35 35 35 35 21 21 21 21 21 21
 20 20 20 20 20 20 34 34 34 34 34 30 30 30 30 30 24 24 24 24 24 31 31 31
 31 31 31 19 19 19 19 19 27 27 27 27 27 27 33 33 33 33 33 33 32 32 32 32
 32 32 26 26 26 26 26 17 17 17 17 17 17 16 16 16 16 16 16 28 28 28 28 28
 29 29 29 29 29 29 11 11 11 11 11 11 10 10 10 10 10 10 12 12 12 12 12 12
 13 13 13 13 13  8  8  8  8  8  8  9  9  9  9  9  9] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Fitting 28 folds for each of 150 candidates, totalling 4200 fits
[CV 1/28] END C=820.33748331373, class_weight=balanced, gamma=0.000200605435179173, kernel=linear; total time=   1.2s
[CV 2/28] END C=820.33748331373, class_weight=balanced, gamma=0.000200605435179173, kernel=linear; total time=   0.5s
[CV 3/28] END C=820.33748331373, class_weight=balanced, gamma=0.000200605435179173, kernel=linear; total time=   0.1s
[CV 4/28] END C=820.33748331373, class_weight=balanced, gamma=0.000200605435179173, kernel=linear; total time=   0.5s
[CV 5/28] END C=820.33748331373, class_weight=balanced, gamma=0.000200605435179173, kernel=linear; total time=   0.2s
[CV 6/28] END C=820.33748331373, class_weight=balanced, gamma=0.000200605435179173, kernel=linear; total time=   0.2s
[CV 7/28] END C=820.33748331373, class_weight=balanced, gamma=0.000200605435179173, kernel=linear; total time=   0.3s
[CV 8/28] END C=820.33748331373, class_weight=balanced, gamma=0.000200605435179173, kernel=linear; total time

[CV 4/28] END C=12.37244304363352, class_weight=None, gamma=0.00015578094797583352, kernel=linear; total time=   0.0s
[CV 5/28] END C=12.37244304363352, class_weight=None, gamma=0.00015578094797583352, kernel=linear; total time=   0.0s
[CV 6/28] END C=12.37244304363352, class_weight=None, gamma=0.00015578094797583352, kernel=linear; total time=   0.0s
[CV 7/28] END C=12.37244304363352, class_weight=None, gamma=0.00015578094797583352, kernel=linear; total time=   0.0s
[CV 8/28] END C=12.37244304363352, class_weight=None, gamma=0.00015578094797583352, kernel=linear; total time=   0.0s
[CV 9/28] END C=12.37244304363352, class_weight=None, gamma=0.00015578094797583352, kernel=linear; total time=   0.0s
[CV 10/28] END C=12.37244304363352, class_weight=None, gamma=0.00015578094797583352, kernel=linear; total time=   0.0s
[CV 11/28] END C=12.37244304363352, class_weight=None, gamma=0.00015578094797583352, kernel=linear; total time=   0.0s
[CV 12/28] END C=12.37244304363352, class_weight=None,

[CV 24/28] END C=70.42509860180209, class_weight=balanced, gamma=0.00010420349869427506, kernel=linear; total time=   0.0s
[CV 25/28] END C=70.42509860180209, class_weight=balanced, gamma=0.00010420349869427506, kernel=linear; total time=   0.0s
[CV 26/28] END C=70.42509860180209, class_weight=balanced, gamma=0.00010420349869427506, kernel=linear; total time=   0.0s
[CV 27/28] END C=70.42509860180209, class_weight=balanced, gamma=0.00010420349869427506, kernel=linear; total time=   0.0s
[CV 28/28] END C=70.42509860180209, class_weight=balanced, gamma=0.00010420349869427506, kernel=linear; total time=   0.0s
[CV 1/28] END C=31.44418779258701, class_weight=None, gamma=0.00023867767939213054, kernel=linear; total time=   0.0s
[CV 2/28] END C=31.44418779258701, class_weight=None, gamma=0.00023867767939213054, kernel=linear; total time=   0.0s
[CV 3/28] END C=31.44418779258701, class_weight=None, gamma=0.00023867767939213054, kernel=linear; total time=   0.0s
[CV 4/28] END C=31.444187792587

[CV 16/28] END C=67.55727813497725, class_weight=balanced, gamma=0.0006644406584318964, kernel=linear; total time=   0.0s
[CV 17/28] END C=67.55727813497725, class_weight=balanced, gamma=0.0006644406584318964, kernel=linear; total time=   0.0s
[CV 18/28] END C=67.55727813497725, class_weight=balanced, gamma=0.0006644406584318964, kernel=linear; total time=   0.0s
[CV 19/28] END C=67.55727813497725, class_weight=balanced, gamma=0.0006644406584318964, kernel=linear; total time=   0.0s
[CV 20/28] END C=67.55727813497725, class_weight=balanced, gamma=0.0006644406584318964, kernel=linear; total time=   0.0s
[CV 21/28] END C=67.55727813497725, class_weight=balanced, gamma=0.0006644406584318964, kernel=linear; total time=   0.0s
[CV 22/28] END C=67.55727813497725, class_weight=balanced, gamma=0.0006644406584318964, kernel=linear; total time=   0.0s
[CV 23/28] END C=67.55727813497725, class_weight=balanced, gamma=0.0006644406584318964, kernel=linear; total time=   0.0s
[CV 24/28] END C=67.5572

...

[CV 12/28] END C=13.626601902747728, class_weight=balanced, gamma=0.0008914503632726238, kernel=linear; total time=   0.0s
[CV 13/28] END C=13.626601902747728, class_weight=balanced, gamma=0.0008914503632726238, kernel=linear; total time=   0.0s
[CV 14/28] END C=13.626601902747728, class_weight=balanced, gamma=0.0008914503632726238, kernel=linear; total time=   0.0s
[CV 15/28] END C=13.626601902747728, class_weight=balanced, gamma=0.0008914503632726238, kernel=linear; total time=   0.0s
[CV 16/28] END C=13.626601902747728, class_weight=balanced, gamma=0.0008914503632726238, kernel=linear; total time=   0.0s
[CV 17/28] END C=13.626601902747728, class_weight=balanced, gamma=0.0008914503632726238, kernel=linear; total time=   0.0s
[CV 18/28] END C=13.626601902747728, class_weight=balanced, gamma=0.0008914503632726238, kernel=linear; total time=   0.0s
[CV 19/28] END C=13.626601902747728, class_weight=balanced, gamma=0.0008914503632726238, kernel=linear; total time=   0.0s
[CV 20/28] END C

[CV 25/28] END C=509.80816408225826, class_weight=None, gamma=0.00037519208269664835, kernel=linear; total time=   0.1s
[CV 26/28] END C=509.80816408225826, class_weight=None, gamma=0.00037519208269664835, kernel=linear; total time=   0.1s
[CV 27/28] END C=509.80816408225826, class_weight=None, gamma=0.00037519208269664835, kernel=linear; total time=   0.2s
[CV 28/28] END C=509.80816408225826, class_weight=None, gamma=0.00037519208269664835, kernel=linear; total time=   0.2s
[CV 1/28] END C=2.802395771739241, class_weight=balanced, gamma=0.00021519422648591765, kernel=linear; total time=   0.0s
[CV 2/28] END C=2.802395771739241, class_weight=balanced, gamma=0.00021519422648591765, kernel=linear; total time=   0.0s
[CV 3/28] END C=2.802395771739241, class_weight=balanced, gamma=0.00021519422648591765, kernel=linear; total time=   0.0s
[CV 4/28] END C=2.802395771739241, class_weight=balanced, gamma=0.00021519422648591765, kernel=linear; total time=   0.0s
[CV 5/28] END C=2.80239577173924

[CV 10/28] END C=236.60675530613858, class_weight=None, gamma=0.00015243000238338313, kernel=linear; total time=   0.0s
[CV 11/28] END C=236.60675530613858, class_weight=None, gamma=0.00015243000238338313, kernel=linear; total time=   0.0s
[CV 12/28] END C=236.60675530613858, class_weight=None, gamma=0.00015243000238338313, kernel=linear; total time=   0.0s
[CV 13/28] END C=236.60675530613858, class_weight=None, gamma=0.00015243000238338313, kernel=linear; total time=   0.1s
[CV 14/28] END C=236.60675530613858, class_weight=None, gamma=0.00015243000238338313, kernel=linear; total time=   0.0s
[CV 15/28] END C=236.60675530613858, class_weight=None, gamma=0.00015243000238338313, kernel=linear; total time=   0.0s
[CV 16/28] END C=236.60675530613858, class_weight=None, gamma=0.00015243000238338313, kernel=linear; total time=   0.0s
[CV 17/28] END C=236.60675530613858, class_weight=None, gamma=0.00015243000238338313, kernel=linear; total time=   0.1s
[CV 18/28] END C=236.60675530613858, cla

[CV 26/28] END C=362.19318441024456, class_weight=None, gamma=0.0001254069380982628, kernel=linear; total time=   0.1s
[CV 27/28] END C=362.19318441024456, class_weight=None, gamma=0.0001254069380982628, kernel=linear; total time=   0.1s
[CV 28/28] END C=362.19318441024456, class_weight=None, gamma=0.0001254069380982628, kernel=linear; total time=   0.4s
[CV 1/28] END C=13.629360906484303, class_weight=balanced, gamma=0.00014061985212423337, kernel=linear; total time=   0.0s
[CV 2/28] END C=13.629360906484303, class_weight=balanced, gamma=0.00014061985212423337, kernel=linear; total time=   0.0s
[CV 3/28] END C=13.629360906484303, class_weight=balanced, gamma=0.00014061985212423337, kernel=linear; total time=   0.0s
[CV 4/28] END C=13.629360906484303, class_weight=balanced, gamma=0.00014061985212423337, kernel=linear; total time=   0.0s
[CV 5/28] END C=13.629360906484303, class_weight=balanced, gamma=0.00014061985212423337, kernel=linear; total time=   0.0s
[CV 6/28] END C=13.629360906

[CV 1/28] END C=519.9024390272451, class_weight=balanced, gamma=0.00010420202071972002, kernel=linear; total time=   0.1s
[CV 2/28] END C=519.9024390272451, class_weight=balanced, gamma=0.00010420202071972002, kernel=linear; total time=   0.4s
[CV 3/28] END C=519.9024390272451, class_weight=balanced, gamma=0.00010420202071972002, kernel=linear; total time=   0.1s
[CV 4/28] END C=519.9024390272451, class_weight=balanced, gamma=0.00010420202071972002, kernel=linear; total time=   0.3s
[CV 5/28] END C=519.9024390272451, class_weight=balanced, gamma=0.00010420202071972002, kernel=linear; total time=   0.1s
[CV 6/28] END C=519.9024390272451, class_weight=balanced, gamma=0.00010420202071972002, kernel=linear; total time=   0.1s
[CV 7/28] END C=519.9024390272451, class_weight=balanced, gamma=0.00010420202071972002, kernel=linear; total time=   0.1s
[CV 8/28] END C=519.9024390272451, class_weight=balanced, gamma=0.00010420202071972002, kernel=linear; total time=   0.2s
[CV 9/28] END C=519.9024

## Hyperparameter tuning for non Linear SVM

In [12]:
param_dist = {
    "C": loguniform(1e0, 1e3),
    "gamma": loguniform(1e-4, 1e-3),
    "kernel": ["rbf"],
    "class_weight": ["balanced", None],
}

n_iter_search = 150

random_search = RandomizedSearchCV(
    SVC(), param_distributions=param_dist, n_iter=n_iter_search, verbose=5, cv=logo
)

In [13]:
start = time()
random_search.fit(X, y, gp)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


report(random_search.cv_results_)

Fitting 28 folds for each of 150 candidates, totalling 4200 fits
[CV 1/28] END C=2.8985097056629745, class_weight=balanced, gamma=0.00010503680286438247, kernel=rbf; total time=   0.0s
[CV 2/28] END C=2.8985097056629745, class_weight=balanced, gamma=0.00010503680286438247, kernel=rbf; total time=   0.0s
[CV 3/28] END C=2.8985097056629745, class_weight=balanced, gamma=0.00010503680286438247, kernel=rbf; total time=   0.0s
[CV 4/28] END C=2.8985097056629745, class_weight=balanced, gamma=0.00010503680286438247, kernel=rbf; total time=   0.0s
[CV 5/28] END C=2.8985097056629745, class_weight=balanced, gamma=0.00010503680286438247, kernel=rbf; total time=   0.0s
[CV 6/28] END C=2.8985097056629745, class_weight=balanced, gamma=0.00010503680286438247, kernel=rbf; total time=   0.0s
[CV 7/28] END C=2.8985097056629745, class_weight=balanced, gamma=0.00010503680286438247, kernel=rbf; total time=   0.0s
[CV 8/28] END C=2.8985097056629745, class_weight=balanced, gamma=0.00010503680286438247, kernel

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=[ 4  4  4  4  4  4  5  5  5  5  5  5  7  7  7  7  7  7  6  6  6  6  6  6
 22 22 22 22 22 22 23 23 23 23 23 23 35 35 35 35 35 35 21 21 21 21 21 21
 20 20 20 20 20 20 34 34 34 34 34 30 30 30 30 30 24 24 24 24 24 31 31 31
 31 31 31 19 19 19 19 19 27 27 27 27 27 27 33 33 33 33 33 33 32 32 32 32
 32 32 26 26 26 26 26 17 17 17 17 17 17 16 16 16 16 16 16 28 28 28 28 28
 29 29 29 29 29 29 11 11 11 11 11 11 10 10 10 10 10 10 12 12 12 12 12 12
 13 13 13 13 13  8  8  8  8  8  8  9  9  9  9  9  9] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


[CV 2/28] END C=12.960414354919173, class_weight=None, gamma=0.00013162340006233405, kernel=rbf; total time=   0.0s
[CV 3/28] END C=12.960414354919173, class_weight=None, gamma=0.00013162340006233405, kernel=rbf; total time=   0.0s
[CV 4/28] END C=12.960414354919173, class_weight=None, gamma=0.00013162340006233405, kernel=rbf; total time=   0.0s
[CV 5/28] END C=12.960414354919173, class_weight=None, gamma=0.00013162340006233405, kernel=rbf; total time=   0.0s
[CV 6/28] END C=12.960414354919173, class_weight=None, gamma=0.00013162340006233405, kernel=rbf; total time=   0.0s
[CV 7/28] END C=12.960414354919173, class_weight=None, gamma=0.00013162340006233405, kernel=rbf; total time=   0.0s
[CV 8/28] END C=12.960414354919173, class_weight=None, gamma=0.00013162340006233405, kernel=rbf; total time=   0.0s
[CV 9/28] END C=12.960414354919173, class_weight=None, gamma=0.00013162340006233405, kernel=rbf; total time=   0.0s
[CV 10/28] END C=12.960414354919173, class_weight=None, gamma=0.00013162

[CV 17/28] END C=1.4576829007135426, class_weight=balanced, gamma=0.0001326511804722507, kernel=rbf; total time=   0.0s
[CV 18/28] END C=1.4576829007135426, class_weight=balanced, gamma=0.0001326511804722507, kernel=rbf; total time=   0.0s
[CV 19/28] END C=1.4576829007135426, class_weight=balanced, gamma=0.0001326511804722507, kernel=rbf; total time=   0.0s
[CV 20/28] END C=1.4576829007135426, class_weight=balanced, gamma=0.0001326511804722507, kernel=rbf; total time=   0.0s
[CV 21/28] END C=1.4576829007135426, class_weight=balanced, gamma=0.0001326511804722507, kernel=rbf; total time=   0.0s
[CV 22/28] END C=1.4576829007135426, class_weight=balanced, gamma=0.0001326511804722507, kernel=rbf; total time=   0.0s
[CV 23/28] END C=1.4576829007135426, class_weight=balanced, gamma=0.0001326511804722507, kernel=rbf; total time=   0.0s
[CV 24/28] END C=1.4576829007135426, class_weight=balanced, gamma=0.0001326511804722507, kernel=rbf; total time=   0.0s
[CV 25/28] END C=1.4576829007135426, cla

[CV 18/28] END C=704.3153801257207, class_weight=None, gamma=0.0008326787194289269, kernel=rbf; total time=   0.0s
[CV 19/28] END C=704.3153801257207, class_weight=None, gamma=0.0008326787194289269, kernel=rbf; total time=   0.0s
[CV 20/28] END C=704.3153801257207, class_weight=None, gamma=0.0008326787194289269, kernel=rbf; total time=   0.0s
[CV 21/28] END C=704.3153801257207, class_weight=None, gamma=0.0008326787194289269, kernel=rbf; total time=   0.0s
[CV 22/28] END C=704.3153801257207, class_weight=None, gamma=0.0008326787194289269, kernel=rbf; total time=   0.0s
[CV 23/28] END C=704.3153801257207, class_weight=None, gamma=0.0008326787194289269, kernel=rbf; total time=   0.0s
[CV 24/28] END C=704.3153801257207, class_weight=None, gamma=0.0008326787194289269, kernel=rbf; total time=   0.0s
[CV 25/28] END C=704.3153801257207, class_weight=None, gamma=0.0008326787194289269, kernel=rbf; total time=   0.0s
[CV 26/28] END C=704.3153801257207, class_weight=None, gamma=0.00083267871942892

...

[CV 15/28] END C=3.3102139996847435, class_weight=None, gamma=0.00015691175630305534, kernel=rbf; total time=   0.0s
[CV 16/28] END C=3.3102139996847435, class_weight=None, gamma=0.00015691175630305534, kernel=rbf; total time=   0.0s
[CV 17/28] END C=3.3102139996847435, class_weight=None, gamma=0.00015691175630305534, kernel=rbf; total time=   0.0s
[CV 18/28] END C=3.3102139996847435, class_weight=None, gamma=0.00015691175630305534, kernel=rbf; total time=   0.0s
[CV 19/28] END C=3.3102139996847435, class_weight=None, gamma=0.00015691175630305534, kernel=rbf; total time=   0.0s
[CV 20/28] END C=3.3102139996847435, class_weight=None, gamma=0.00015691175630305534, kernel=rbf; total time=   0.0s
[CV 21/28] END C=3.3102139996847435, class_weight=None, gamma=0.00015691175630305534, kernel=rbf; total time=   0.0s
[CV 22/28] END C=3.3102139996847435, class_weight=None, gamma=0.00015691175630305534, kernel=rbf; total time=   0.0s
[CV 23/28] END C=3.3102139996847435, class_weight=None, gamma=0.

[CV 5/28] END C=1.7467064750231964, class_weight=balanced, gamma=0.0002859677439670634, kernel=rbf; total time=   0.0s
[CV 6/28] END C=1.7467064750231964, class_weight=balanced, gamma=0.0002859677439670634, kernel=rbf; total time=   0.0s
[CV 7/28] END C=1.7467064750231964, class_weight=balanced, gamma=0.0002859677439670634, kernel=rbf; total time=   0.0s
[CV 8/28] END C=1.7467064750231964, class_weight=balanced, gamma=0.0002859677439670634, kernel=rbf; total time=   0.0s
[CV 9/28] END C=1.7467064750231964, class_weight=balanced, gamma=0.0002859677439670634, kernel=rbf; total time=   0.0s
[CV 10/28] END C=1.7467064750231964, class_weight=balanced, gamma=0.0002859677439670634, kernel=rbf; total time=   0.0s
[CV 11/28] END C=1.7467064750231964, class_weight=balanced, gamma=0.0002859677439670634, kernel=rbf; total time=   0.0s
[CV 12/28] END C=1.7467064750231964, class_weight=balanced, gamma=0.0002859677439670634, kernel=rbf; total time=   0.0s
[CV 13/28] END C=1.7467064750231964, class_we

[CV 24/28] END C=380.4910196214689, class_weight=None, gamma=0.0002344565660227688, kernel=rbf; total time=   0.0s
[CV 25/28] END C=380.4910196214689, class_weight=None, gamma=0.0002344565660227688, kernel=rbf; total time=   0.0s
[CV 26/28] END C=380.4910196214689, class_weight=None, gamma=0.0002344565660227688, kernel=rbf; total time=   0.0s
[CV 27/28] END C=380.4910196214689, class_weight=None, gamma=0.0002344565660227688, kernel=rbf; total time=   0.0s
[CV 28/28] END C=380.4910196214689, class_weight=None, gamma=0.0002344565660227688, kernel=rbf; total time=   0.0s
[CV 1/28] END C=45.45014363769032, class_weight=balanced, gamma=0.00010968172915404976, kernel=rbf; total time=   0.0s
[CV 2/28] END C=45.45014363769032, class_weight=balanced, gamma=0.00010968172915404976, kernel=rbf; total time=   0.0s
[CV 3/28] END C=45.45014363769032, class_weight=balanced, gamma=0.00010968172915404976, kernel=rbf; total time=   0.0s
[CV 4/28] END C=45.45014363769032, class_weight=balanced, gamma=0.00

[CV 4/28] END C=11.755323860208959, class_weight=balanced, gamma=0.0005072525769712272, kernel=rbf; total time=   0.0s
[CV 5/28] END C=11.755323860208959, class_weight=balanced, gamma=0.0005072525769712272, kernel=rbf; total time=   0.0s
[CV 6/28] END C=11.755323860208959, class_weight=balanced, gamma=0.0005072525769712272, kernel=rbf; total time=   0.0s
[CV 7/28] END C=11.755323860208959, class_weight=balanced, gamma=0.0005072525769712272, kernel=rbf; total time=   0.0s
[CV 8/28] END C=11.755323860208959, class_weight=balanced, gamma=0.0005072525769712272, kernel=rbf; total time=   0.0s
[CV 9/28] END C=11.755323860208959, class_weight=balanced, gamma=0.0005072525769712272, kernel=rbf; total time=   0.0s
[CV 10/28] END C=11.755323860208959, class_weight=balanced, gamma=0.0005072525769712272, kernel=rbf; total time=   0.0s
[CV 11/28] END C=11.755323860208959, class_weight=balanced, gamma=0.0005072525769712272, kernel=rbf; total time=   0.0s
[CV 12/28] END C=11.755323860208959, class_wei

[CV 17/28] END C=358.6289090542484, class_weight=balanced, gamma=0.00041933240520437983, kernel=rbf; total time=   0.0s
[CV 18/28] END C=358.6289090542484, class_weight=balanced, gamma=0.00041933240520437983, kernel=rbf; total time=   0.0s
[CV 19/28] END C=358.6289090542484, class_weight=balanced, gamma=0.00041933240520437983, kernel=rbf; total time=   0.0s
[CV 20/28] END C=358.6289090542484, class_weight=balanced, gamma=0.00041933240520437983, kernel=rbf; total time=   0.0s
[CV 21/28] END C=358.6289090542484, class_weight=balanced, gamma=0.00041933240520437983, kernel=rbf; total time=   0.0s
[CV 22/28] END C=358.6289090542484, class_weight=balanced, gamma=0.00041933240520437983, kernel=rbf; total time=   0.0s
[CV 23/28] END C=358.6289090542484, class_weight=balanced, gamma=0.00041933240520437983, kernel=rbf; total time=   0.0s
[CV 24/28] END C=358.6289090542484, class_weight=balanced, gamma=0.00041933240520437983, kernel=rbf; total time=   0.0s
[CV 25/28] END C=358.6289090542484, clas

# Stress Dataset

In [15]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML-Project/Sliiding_method_Data/acute_stress/no_overlap/normalized')

li = []
for filename in filenames:
    dataframe=pd.read_csv(os.path.join(r'/Users/apostoloskalatzis/Downloads/ML-Project/Sliiding_method_Data/acute_stress/no_overlap/normalized',filename))
    li.append(dataframe)

data = pd.concat(li, axis=0, ignore_index=True)

In [16]:
columns = [ 'HR_mean', 'HRV_RMSSD','HRV_MeanNN',  'HRV_LFHF', 'RSP_mean', 'RRV_RMSSD', 'RRV_MeanBB']

In [17]:
gp = data["ID"].values
data1= data[columns]
X=data1[columns]
y=data['label']

In [18]:
logo = LeaveOneGroupOut()
logo.get_n_splits(X, y, gp)

25

## Hyperparameter tuning for Random Forest

In [19]:
# GAN parameters

param_dist = {
    "max_depth": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    "max_features": ['auto', 'sqrt'],
    "bootstrap": [True, False],
    "min_samples_leaf": [1, 2, 4],
    "min_samples_split": [2, 5, 10],
    "n_estimators": [50,100, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

n_iter_search = 150
random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=n_iter_search,verbose=5, cv=logo
)

In [20]:
start = time()
random_search.fit(X, y, gp)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


report(random_search.cv_results_)

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=[ 4  4  4  4  4  4  4  5  5  5  5  5  5  5  7  7  7  7  7  7  7  6  6  6
  6  6  6  6  2  2  2  2  2  2  2  3  3  3  3  3  3  3  1  1  1  1  1  1
  1 22 22 22 22 22 22 22 22 23 23 23 23 23 23 23 21 21 21 21 21 21 21 20
 20 20 20 20 20 20 30 30 30 30 30 30 30 24 24 24 24 24 24 25 25 25 25 25
 25 31 31 31 31 31 31 33 33 33 33 33 33 33 32 32 32 32 32 32 26 26 26 26
 26 26 26 17 17 17 17 17 17 17 14 14 14 14 14 14 14 28 28 28 28 28 28 29
 29 29 29 29 29 29 12 12 12 12 12 12 13 13 13 13 13 13 13 13  8  8  8  8
  8  8  8] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Fitting 25 folds for each of 150 candidates, totalling 3750 fits
[CV 1/25] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.6s
[CV 2/25] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.5s
[CV 3/25] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.5s
[CV 4/25] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.5s
[CV 5/25] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.5s
[CV 6/25] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.5s
[CV 7/25] END bootstrap=False, max_depth=10, max_features=auto, min_samples_l

[CV 9/25] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 10/25] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 11/25] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 12/25] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 13/25] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.5s
[CV 14/25] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.5s
[CV 15/25] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s


[CV 17/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   0.7s
[CV 18/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   0.6s
[CV 19/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   0.6s
[CV 20/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   0.6s
[CV 21/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   0.6s
[CV 22/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   0.6s
[CV 23/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=

[CV 24/25] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=1200; total time=   0.6s
[CV 25/25] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=1200; total time=   0.6s
[CV 1/25] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 2/25] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 3/25] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 4/25] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 5/25] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.5s
[CV 6/2

...

[CV 19/25] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   0.4s
[CV 20/25] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   0.4s
[CV 21/25] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   0.4s
[CV 22/25] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   0.4s
[CV 23/25] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   0.4s
[CV 24/25] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   0.4s
[CV 25/25] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   0.4s

[CV 2/25] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   0.2s
[CV 3/25] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   0.2s
[CV 4/25] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   0.2s
[CV 5/25] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   0.2s
[CV 6/25] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   0.2s
[CV 7/25] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   0.2s
[CV 8/25] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   0.2s

[CV 10/25] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 11/25] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 12/25] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 13/25] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 14/25] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 15/25] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 16/25] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=

[CV 17/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=800; total time=   0.4s
[CV 18/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=800; total time=   0.4s
[CV 19/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=800; total time=   0.4s
[CV 20/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=800; total time=   0.4s
[CV 21/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=800; total time=   0.4s
[CV 22/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=800; total time=   0.4s
[CV 23/25] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=800; total time=

[CV 24/25] END bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   0.7s
[CV 25/25] END bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   0.7s
[CV 1/25] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   1.3s
[CV 2/25] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   1.4s
[CV 3/25] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   1.3s
[CV 4/25] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   1.4s
[CV 5/25] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   

## Hyperparameter tunning for Linear SVM 

In [21]:
param_dist = {
    "C": loguniform(1e0, 1e3),
    "gamma": loguniform(1e-4, 1e-3),
    "kernel": ["linear"],
    "class_weight": ["balanced", None],
}

n_iter_search = 150

random_search = RandomizedSearchCV(
    SVC(), param_distributions=param_dist, n_iter=n_iter_search, verbose=5, cv=logo
)

In [22]:
start = time()
random_search.fit(X, y, gp)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


report(random_search.cv_results_)

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=[ 4  4  4  4  4  4  4  5  5  5  5  5  5  5  7  7  7  7  7  7  7  6  6  6
  6  6  6  6  2  2  2  2  2  2  2  3  3  3  3  3  3  3  1  1  1  1  1  1
  1 22 22 22 22 22 22 22 22 23 23 23 23 23 23 23 21 21 21 21 21 21 21 20
 20 20 20 20 20 20 30 30 30 30 30 30 30 24 24 24 24 24 24 25 25 25 25 25
 25 31 31 31 31 31 31 33 33 33 33 33 33 33 32 32 32 32 32 32 26 26 26 26
 26 26 26 17 17 17 17 17 17 17 14 14 14 14 14 14 14 28 28 28 28 28 28 29
 29 29 29 29 29 29 12 12 12 12 12 12 13 13 13 13 13 13 13 13  8  8  8  8
  8  8  8] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Fitting 25 folds for each of 150 candidates, totalling 3750 fits
[CV 1/25] END C=158.5160510352917, class_weight=balanced, gamma=0.00018151514649346613, kernel=linear; total time=   0.0s
[CV 2/25] END C=158.5160510352917, class_weight=balanced, gamma=0.00018151514649346613, kernel=linear; total time=   0.0s
[CV 3/25] END C=158.5160510352917, class_weight=balanced, gamma=0.00018151514649346613, kernel=linear; total time=   0.0s
[CV 4/25] END C=158.5160510352917, class_weight=balanced, gamma=0.00018151514649346613, kernel=linear; total time=   0.0s
[CV 5/25] END C=158.5160510352917, class_weight=balanced, gamma=0.00018151514649346613, kernel=linear; total time=   0.0s
[CV 6/25] END C=158.5160510352917, class_weight=balanced, gamma=0.00018151514649346613, kernel=linear; total time=   0.0s
[CV 7/25] END C=158.5160510352917, class_weight=balanced, gamma=0.00018151514649346613, kernel=linear; total time=   0.0s
[CV 8/25] END C=158.5160510352917, class_weight=balanced, gamma=0.000181515146493

[CV 7/25] END C=96.71908925043147, class_weight=None, gamma=0.0008738154753017217, kernel=linear; total time=   0.0s
[CV 8/25] END C=96.71908925043147, class_weight=None, gamma=0.0008738154753017217, kernel=linear; total time=   0.0s
[CV 9/25] END C=96.71908925043147, class_weight=None, gamma=0.0008738154753017217, kernel=linear; total time=   0.0s
[CV 10/25] END C=96.71908925043147, class_weight=None, gamma=0.0008738154753017217, kernel=linear; total time=   0.0s
[CV 11/25] END C=96.71908925043147, class_weight=None, gamma=0.0008738154753017217, kernel=linear; total time=   0.0s
[CV 12/25] END C=96.71908925043147, class_weight=None, gamma=0.0008738154753017217, kernel=linear; total time=   0.0s
[CV 13/25] END C=96.71908925043147, class_weight=None, gamma=0.0008738154753017217, kernel=linear; total time=   0.0s
[CV 14/25] END C=96.71908925043147, class_weight=None, gamma=0.0008738154753017217, kernel=linear; total time=   0.0s
[CV 15/25] END C=96.71908925043147, class_weight=None, gamm

[CV 17/25] END C=4.760814778907761, class_weight=balanced, gamma=0.00029639468105786343, kernel=linear; total time=   0.0s
[CV 18/25] END C=4.760814778907761, class_weight=balanced, gamma=0.00029639468105786343, kernel=linear; total time=   0.0s
[CV 19/25] END C=4.760814778907761, class_weight=balanced, gamma=0.00029639468105786343, kernel=linear; total time=   0.0s
[CV 20/25] END C=4.760814778907761, class_weight=balanced, gamma=0.00029639468105786343, kernel=linear; total time=   0.0s
[CV 21/25] END C=4.760814778907761, class_weight=balanced, gamma=0.00029639468105786343, kernel=linear; total time=   0.0s
[CV 22/25] END C=4.760814778907761, class_weight=balanced, gamma=0.00029639468105786343, kernel=linear; total time=   0.0s
[CV 23/25] END C=4.760814778907761, class_weight=balanced, gamma=0.00029639468105786343, kernel=linear; total time=   0.0s
[CV 24/25] END C=4.760814778907761, class_weight=balanced, gamma=0.00029639468105786343, kernel=linear; total time=   0.0s
[CV 25/25] END C

[CV 1/25] END C=355.6746440598981, class_weight=balanced, gamma=0.0001884897424019801, kernel=linear; total time=   0.0s
[CV 2/25] END C=355.6746440598981, class_weight=balanced, gamma=0.0001884897424019801, kernel=linear; total time=   0.0s
[CV 3/25] END C=355.6746440598981, class_weight=balanced, gamma=0.0001884897424019801, kernel=linear; total time=   0.0s
[CV 4/25] END C=355.6746440598981, class_weight=balanced, gamma=0.0001884897424019801, kernel=linear; total time=   0.0s
[CV 5/25] END C=355.6746440598981, class_weight=balanced, gamma=0.0001884897424019801, kernel=linear; total time=   0.0s
[CV 6/25] END C=355.6746440598981, class_weight=balanced, gamma=0.0001884897424019801, kernel=linear; total time=   0.0s
[CV 7/25] END C=355.6746440598981, class_weight=balanced, gamma=0.0001884897424019801, kernel=linear; total time=   0.0s
[CV 8/25] END C=355.6746440598981, class_weight=balanced, gamma=0.0001884897424019801, kernel=linear; total time=   0.0s
[CV 9/25] END C=355.674644059898

...

[CV 9/25] END C=3.838362349355641, class_weight=None, gamma=0.00039118582206832054, kernel=linear; total time=   0.0s
[CV 10/25] END C=3.838362349355641, class_weight=None, gamma=0.00039118582206832054, kernel=linear; total time=   0.0s
[CV 11/25] END C=3.838362349355641, class_weight=None, gamma=0.00039118582206832054, kernel=linear; total time=   0.0s
[CV 12/25] END C=3.838362349355641, class_weight=None, gamma=0.00039118582206832054, kernel=linear; total time=   0.0s
[CV 13/25] END C=3.838362349355641, class_weight=None, gamma=0.00039118582206832054, kernel=linear; total time=   0.0s
[CV 14/25] END C=3.838362349355641, class_weight=None, gamma=0.00039118582206832054, kernel=linear; total time=   0.0s
[CV 15/25] END C=3.838362349355641, class_weight=None, gamma=0.00039118582206832054, kernel=linear; total time=   0.0s
[CV 16/25] END C=3.838362349355641, class_weight=None, gamma=0.00039118582206832054, kernel=linear; total time=   0.0s
[CV 17/25] END C=3.838362349355641, class_weight=

[CV 2/25] END C=5.616036860409238, class_weight=None, gamma=0.00026015313375930295, kernel=linear; total time=   0.0s
[CV 3/25] END C=5.616036860409238, class_weight=None, gamma=0.00026015313375930295, kernel=linear; total time=   0.0s
[CV 4/25] END C=5.616036860409238, class_weight=None, gamma=0.00026015313375930295, kernel=linear; total time=   0.0s
[CV 5/25] END C=5.616036860409238, class_weight=None, gamma=0.00026015313375930295, kernel=linear; total time=   0.0s
[CV 6/25] END C=5.616036860409238, class_weight=None, gamma=0.00026015313375930295, kernel=linear; total time=   0.0s
[CV 7/25] END C=5.616036860409238, class_weight=None, gamma=0.00026015313375930295, kernel=linear; total time=   0.0s
[CV 8/25] END C=5.616036860409238, class_weight=None, gamma=0.00026015313375930295, kernel=linear; total time=   0.0s
[CV 9/25] END C=5.616036860409238, class_weight=None, gamma=0.00026015313375930295, kernel=linear; total time=   0.0s
[CV 10/25] END C=5.616036860409238, class_weight=None, g

[CV 10/25] END C=238.84242658474133, class_weight=None, gamma=0.0008161028190514906, kernel=linear; total time=   0.0s
[CV 11/25] END C=238.84242658474133, class_weight=None, gamma=0.0008161028190514906, kernel=linear; total time=   0.0s
[CV 12/25] END C=238.84242658474133, class_weight=None, gamma=0.0008161028190514906, kernel=linear; total time=   0.0s
[CV 13/25] END C=238.84242658474133, class_weight=None, gamma=0.0008161028190514906, kernel=linear; total time=   0.0s
[CV 14/25] END C=238.84242658474133, class_weight=None, gamma=0.0008161028190514906, kernel=linear; total time=   0.0s
[CV 15/25] END C=238.84242658474133, class_weight=None, gamma=0.0008161028190514906, kernel=linear; total time=   0.0s
[CV 16/25] END C=238.84242658474133, class_weight=None, gamma=0.0008161028190514906, kernel=linear; total time=   0.0s
[CV 17/25] END C=238.84242658474133, class_weight=None, gamma=0.0008161028190514906, kernel=linear; total time=   0.0s
[CV 18/25] END C=238.84242658474133, class_weigh

[CV 25/25] END C=5.812754280082337, class_weight=None, gamma=0.0004183048884549958, kernel=linear; total time=   0.0s
[CV 1/25] END C=220.42486606427403, class_weight=None, gamma=0.0001050904157848754, kernel=linear; total time=   0.0s
[CV 2/25] END C=220.42486606427403, class_weight=None, gamma=0.0001050904157848754, kernel=linear; total time=   0.0s
[CV 3/25] END C=220.42486606427403, class_weight=None, gamma=0.0001050904157848754, kernel=linear; total time=   0.0s
[CV 4/25] END C=220.42486606427403, class_weight=None, gamma=0.0001050904157848754, kernel=linear; total time=   0.0s
[CV 5/25] END C=220.42486606427403, class_weight=None, gamma=0.0001050904157848754, kernel=linear; total time=   0.0s
[CV 6/25] END C=220.42486606427403, class_weight=None, gamma=0.0001050904157848754, kernel=linear; total time=   0.0s
[CV 7/25] END C=220.42486606427403, class_weight=None, gamma=0.0001050904157848754, kernel=linear; total time=   0.0s
[CV 8/25] END C=220.42486606427403, class_weight=None, g

[CV 8/25] END C=555.6089668685908, class_weight=None, gamma=0.0006125772854743744, kernel=linear; total time=   0.0s
[CV 9/25] END C=555.6089668685908, class_weight=None, gamma=0.0006125772854743744, kernel=linear; total time=   0.0s
[CV 10/25] END C=555.6089668685908, class_weight=None, gamma=0.0006125772854743744, kernel=linear; total time=   0.0s
[CV 11/25] END C=555.6089668685908, class_weight=None, gamma=0.0006125772854743744, kernel=linear; total time=   0.0s
[CV 12/25] END C=555.6089668685908, class_weight=None, gamma=0.0006125772854743744, kernel=linear; total time=   0.0s
[CV 13/25] END C=555.6089668685908, class_weight=None, gamma=0.0006125772854743744, kernel=linear; total time=   0.0s
[CV 14/25] END C=555.6089668685908, class_weight=None, gamma=0.0006125772854743744, kernel=linear; total time=   0.0s
[CV 15/25] END C=555.6089668685908, class_weight=None, gamma=0.0006125772854743744, kernel=linear; total time=   0.0s
[CV 16/25] END C=555.6089668685908, class_weight=None, gam

## Hyperparameter tunning for non Linear SVM

In [23]:
param_dist = {
    "C": loguniform(1e0, 1e3),
    "gamma": loguniform(1e-4, 1e-3),
    "kernel": ["rbf"],
    "class_weight": ["balanced", None],
}

n_iter_search = 150

random_search = RandomizedSearchCV(
    SVC(), param_distributions=param_dist, n_iter=n_iter_search, verbose=5, cv=logo
)

In [24]:
start = time()
random_search.fit(X, y, gp)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


report(random_search.cv_results_)

Fitting 25 folds for each of 150 candidates, totalling 3750 fits
[CV 1/25] END C=19.983106609114223, class_weight=balanced, gamma=0.00023575621143526285, kernel=rbf; total time=   0.0s
[CV 2/25] END C=19.983106609114223, class_weight=balanced, gamma=0.00023575621143526285, kernel=rbf; total time=   0.0s
[CV 3/25] END C=19.983106609114223, class_weight=balanced, gamma=0.00023575621143526285, kernel=rbf; total time=   0.0s
[CV 4/25] END C=19.983106609114223, class_weight=balanced, gamma=0.00023575621143526285, kernel=rbf; total time=   0.0s
[CV 5/25] END C=19.983106609114223, class_weight=balanced, gamma=0.00023575621143526285, kernel=rbf; total time=   0.0s
[CV 6/25] END C=19.983106609114223, class_weight=balanced, gamma=0.00023575621143526285, kernel=rbf; total time=   0.0s
[CV 7/25] END C=19.983106609114223, class_weight=balanced, gamma=0.00023575621143526285, kernel=rbf; total time=   0.0s
[CV 8/25] END C=19.983106609114223, class_weight=balanced, gamma=0.00023575621143526285, kernel

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=[ 4  4  4  4  4  4  4  5  5  5  5  5  5  5  7  7  7  7  7  7  7  6  6  6
  6  6  6  6  2  2  2  2  2  2  2  3  3  3  3  3  3  3  1  1  1  1  1  1
  1 22 22 22 22 22 22 22 22 23 23 23 23 23 23 23 21 21 21 21 21 21 21 20
 20 20 20 20 20 20 30 30 30 30 30 30 30 24 24 24 24 24 24 25 25 25 25 25
 25 31 31 31 31 31 31 33 33 33 33 33 33 33 32 32 32 32 32 32 26 26 26 26
 26 26 26 17 17 17 17 17 17 17 14 14 14 14 14 14 14 28 28 28 28 28 28 29
 29 29 29 29 29 29 12 12 12 12 12 12 13 13 13 13 13 13 13 13  8  8  8  8
  8  8  8] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


[CV 9/25] END C=57.68281968052015, class_weight=None, gamma=0.000660638555852118, kernel=rbf; total time=   0.0s
[CV 10/25] END C=57.68281968052015, class_weight=None, gamma=0.000660638555852118, kernel=rbf; total time=   0.0s
[CV 11/25] END C=57.68281968052015, class_weight=None, gamma=0.000660638555852118, kernel=rbf; total time=   0.0s
[CV 12/25] END C=57.68281968052015, class_weight=None, gamma=0.000660638555852118, kernel=rbf; total time=   0.0s
[CV 13/25] END C=57.68281968052015, class_weight=None, gamma=0.000660638555852118, kernel=rbf; total time=   0.0s
[CV 14/25] END C=57.68281968052015, class_weight=None, gamma=0.000660638555852118, kernel=rbf; total time=   0.0s
[CV 15/25] END C=57.68281968052015, class_weight=None, gamma=0.000660638555852118, kernel=rbf; total time=   0.0s
[CV 16/25] END C=57.68281968052015, class_weight=None, gamma=0.000660638555852118, kernel=rbf; total time=   0.0s
[CV 17/25] END C=57.68281968052015, class_weight=None, gamma=0.000660638555852118, kernel

[CV 20/25] END C=37.73952614409301, class_weight=None, gamma=0.00038877249624089984, kernel=rbf; total time=   0.0s
[CV 21/25] END C=37.73952614409301, class_weight=None, gamma=0.00038877249624089984, kernel=rbf; total time=   0.0s
[CV 22/25] END C=37.73952614409301, class_weight=None, gamma=0.00038877249624089984, kernel=rbf; total time=   0.0s
[CV 23/25] END C=37.73952614409301, class_weight=None, gamma=0.00038877249624089984, kernel=rbf; total time=   0.0s
[CV 24/25] END C=37.73952614409301, class_weight=None, gamma=0.00038877249624089984, kernel=rbf; total time=   0.0s
[CV 25/25] END C=37.73952614409301, class_weight=None, gamma=0.00038877249624089984, kernel=rbf; total time=   0.0s
[CV 1/25] END C=8.244616957987978, class_weight=balanced, gamma=0.00011756153069750007, kernel=rbf; total time=   0.0s
[CV 2/25] END C=8.244616957987978, class_weight=balanced, gamma=0.00011756153069750007, kernel=rbf; total time=   0.0s
[CV 3/25] END C=8.244616957987978, class_weight=balanced, gamma=0.

[CV 16/25] END C=14.79830328455842, class_weight=balanced, gamma=0.00014287392817419024, kernel=rbf; total time=   0.0s
[CV 17/25] END C=14.79830328455842, class_weight=balanced, gamma=0.00014287392817419024, kernel=rbf; total time=   0.0s
[CV 18/25] END C=14.79830328455842, class_weight=balanced, gamma=0.00014287392817419024, kernel=rbf; total time=   0.0s
[CV 19/25] END C=14.79830328455842, class_weight=balanced, gamma=0.00014287392817419024, kernel=rbf; total time=   0.0s
[CV 20/25] END C=14.79830328455842, class_weight=balanced, gamma=0.00014287392817419024, kernel=rbf; total time=   0.0s
[CV 21/25] END C=14.79830328455842, class_weight=balanced, gamma=0.00014287392817419024, kernel=rbf; total time=   0.0s
[CV 22/25] END C=14.79830328455842, class_weight=balanced, gamma=0.00014287392817419024, kernel=rbf; total time=   0.0s
[CV 23/25] END C=14.79830328455842, class_weight=balanced, gamma=0.00014287392817419024, kernel=rbf; total time=   0.0s
[CV 24/25] END C=14.79830328455842, clas

...

[CV 9/25] END C=11.143094297924907, class_weight=balanced, gamma=0.0001859250435942281, kernel=rbf; total time=   0.0s
[CV 10/25] END C=11.143094297924907, class_weight=balanced, gamma=0.0001859250435942281, kernel=rbf; total time=   0.0s
[CV 11/25] END C=11.143094297924907, class_weight=balanced, gamma=0.0001859250435942281, kernel=rbf; total time=   0.0s
[CV 12/25] END C=11.143094297924907, class_weight=balanced, gamma=0.0001859250435942281, kernel=rbf; total time=   0.0s
[CV 13/25] END C=11.143094297924907, class_weight=balanced, gamma=0.0001859250435942281, kernel=rbf; total time=   0.0s
[CV 14/25] END C=11.143094297924907, class_weight=balanced, gamma=0.0001859250435942281, kernel=rbf; total time=   0.0s
[CV 15/25] END C=11.143094297924907, class_weight=balanced, gamma=0.0001859250435942281, kernel=rbf; total time=   0.0s
[CV 16/25] END C=11.143094297924907, class_weight=balanced, gamma=0.0001859250435942281, kernel=rbf; total time=   0.0s
[CV 17/25] END C=11.143094297924907, clas

[CV 22/25] END C=18.618572135752515, class_weight=balanced, gamma=0.00039335787309740443, kernel=rbf; total time=   0.0s
[CV 23/25] END C=18.618572135752515, class_weight=balanced, gamma=0.00039335787309740443, kernel=rbf; total time=   0.0s
[CV 24/25] END C=18.618572135752515, class_weight=balanced, gamma=0.00039335787309740443, kernel=rbf; total time=   0.0s
[CV 25/25] END C=18.618572135752515, class_weight=balanced, gamma=0.00039335787309740443, kernel=rbf; total time=   0.0s
[CV 1/25] END C=2.0032582789091165, class_weight=None, gamma=0.0007700816570937187, kernel=rbf; total time=   0.0s
[CV 2/25] END C=2.0032582789091165, class_weight=None, gamma=0.0007700816570937187, kernel=rbf; total time=   0.0s
[CV 3/25] END C=2.0032582789091165, class_weight=None, gamma=0.0007700816570937187, kernel=rbf; total time=   0.0s
[CV 4/25] END C=2.0032582789091165, class_weight=None, gamma=0.0007700816570937187, kernel=rbf; total time=   0.0s
[CV 5/25] END C=2.0032582789091165, class_weight=None, g

[CV 17/25] END C=133.41040153657553, class_weight=balanced, gamma=0.0006164433466189068, kernel=rbf; total time=   0.0s
[CV 18/25] END C=133.41040153657553, class_weight=balanced, gamma=0.0006164433466189068, kernel=rbf; total time=   0.0s
[CV 19/25] END C=133.41040153657553, class_weight=balanced, gamma=0.0006164433466189068, kernel=rbf; total time=   0.0s
[CV 20/25] END C=133.41040153657553, class_weight=balanced, gamma=0.0006164433466189068, kernel=rbf; total time=   0.0s
[CV 21/25] END C=133.41040153657553, class_weight=balanced, gamma=0.0006164433466189068, kernel=rbf; total time=   0.0s
[CV 22/25] END C=133.41040153657553, class_weight=balanced, gamma=0.0006164433466189068, kernel=rbf; total time=   0.0s
[CV 23/25] END C=133.41040153657553, class_weight=balanced, gamma=0.0006164433466189068, kernel=rbf; total time=   0.0s
[CV 24/25] END C=133.41040153657553, class_weight=balanced, gamma=0.0006164433466189068, kernel=rbf; total time=   0.0s
[CV 25/25] END C=133.41040153657553, cla

[CV 19/25] END C=61.45655633670077, class_weight=None, gamma=0.00045258963584237234, kernel=rbf; total time=   0.0s
[CV 20/25] END C=61.45655633670077, class_weight=None, gamma=0.00045258963584237234, kernel=rbf; total time=   0.0s
[CV 21/25] END C=61.45655633670077, class_weight=None, gamma=0.00045258963584237234, kernel=rbf; total time=   0.0s
[CV 22/25] END C=61.45655633670077, class_weight=None, gamma=0.00045258963584237234, kernel=rbf; total time=   0.0s
[CV 23/25] END C=61.45655633670077, class_weight=None, gamma=0.00045258963584237234, kernel=rbf; total time=   0.0s
[CV 24/25] END C=61.45655633670077, class_weight=None, gamma=0.00045258963584237234, kernel=rbf; total time=   0.0s
[CV 25/25] END C=61.45655633670077, class_weight=None, gamma=0.00045258963584237234, kernel=rbf; total time=   0.0s
[CV 1/25] END C=41.423609153570844, class_weight=None, gamma=0.0004142161338004012, kernel=rbf; total time=   0.0s
[CV 2/25] END C=41.423609153570844, class_weight=None, gamma=0.0004142161

[CV 23/25] END C=5.459174961856404, class_weight=balanced, gamma=0.0005468886091844648, kernel=rbf; total time=   0.0s
[CV 24/25] END C=5.459174961856404, class_weight=balanced, gamma=0.0005468886091844648, kernel=rbf; total time=   0.0s
[CV 25/25] END C=5.459174961856404, class_weight=balanced, gamma=0.0005468886091844648, kernel=rbf; total time=   0.0s
[CV 1/25] END C=380.32432489978396, class_weight=balanced, gamma=0.000485428238888694, kernel=rbf; total time=   0.0s
[CV 2/25] END C=380.32432489978396, class_weight=balanced, gamma=0.000485428238888694, kernel=rbf; total time=   0.0s
[CV 3/25] END C=380.32432489978396, class_weight=balanced, gamma=0.000485428238888694, kernel=rbf; total time=   0.0s
[CV 4/25] END C=380.32432489978396, class_weight=balanced, gamma=0.000485428238888694, kernel=rbf; total time=   0.0s
[CV 5/25] END C=380.32432489978396, class_weight=balanced, gamma=0.000485428238888694, kernel=rbf; total time=   0.0s
[CV 6/25] END C=380.32432489978396, class_weight=bala